<a href="https://colab.research.google.com/github/AnuragRai017/Python-Projects/blob/master/Chochlate_sale_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import warnings

# Configure settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# --- 1. Load Data ---
try:
    # Try reading with standard UTF-8 first
    df = pd.read_csv('Chocolate Sales.csv')
except UnicodeDecodeError:
    # If that fails, try with utf-8-sig to handle potential BOM (Byte Order Mark)
    df = pd.read_csv('Chocolate Sales.csv', encoding='utf-8-sig')

print("--- Initial Data Head ---")
print(df.head())
print("\n--- Data Info ---")
df.info()
print("\n--- Initial Data Shape ---")
print(df.shape)

# --- 2. Data Cleaning and Preprocessing ---

# Clean column names (remove leading/trailing spaces, replace spaces with underscores)
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Check for missing values
print("\n--- Missing Values ---")
print(df.isnull().sum())
# No missing values detected initially, but cleaning might introduce them if formats are unexpected.

# Clean 'Amount' column
# Remove '$', ',', and spaces, then convert to numeric
# Using errors='coerce' will turn unparseable values into NaN
df['Amount'] = df['Amount'].astype(str).str.replace(r'[$,\s]', '', regex=True)
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Convert 'Date' column to datetime objects
# Use errors='coerce' for robustness
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')

# Convert 'Boxes_Shipped' to numeric (just in case it was read as object)
df['Boxes_Shipped'] = pd.to_numeric(df['Boxes_Shipped'], errors='coerce')

# Check for NaNs introduced during cleaning
print("\n--- Missing Values After Cleaning ---")
print(df.isnull().sum())
# If any NaNs were introduced, decide how to handle them (e.g., drop rows, impute)
# For this example, let's drop rows with NaNs resulting from conversion errors
df.dropna(subset=['Amount', 'Date', 'Boxes_Shipped'], inplace=True)

# Check for duplicates
print("\n--- Duplicate Rows ---")
print(f"Number of duplicate rows: {df.duplicated().sum()}")
# If duplicates exist, decide whether to remove them
df.drop_duplicates(inplace=True)

# Feature Engineering: Extract time components
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month_Name'] = df['Date'].dt.month_name()
df['Day_Name'] = df['Date'].dt.day_name()
df['Week_of_Year'] = df['Date'].dt.isocalendar().week.astype(int) # Use isocalendar().week
df['Quarter'] = df['Date'].dt.quarter

# Feature Engineering: Calculate Price per Box (handle potential division by zero)
df['Price_Per_Box'] = df.apply(lambda row: row['Amount'] / row['Boxes_Shipped'] if row['Boxes_Shipped'] > 0 else 0, axis=1)
# Remove potential outliers/infinite values if Boxes_Shipped was 0 (though we check > 0)
df = df[np.isfinite(df['Price_Per_Box'])]


print("\n--- Data Info After Cleaning & Feature Engineering ---")
df.info()
print("\n--- Data Shape After Cleaning ---")
print(df.shape)
print("\n--- Descriptive Statistics ---")
# Include datetime=False for newer pandas versions if needed
print(df.describe(include='number'))
print("\n--- Descriptive Statistics (Categorical) ---")
print(df.describe(include='object'))


# --- 3. Exploratory Data Analysis (EDA) ---

print("\n--- Starting EDA Visualizations ---")

# 3.1. Distribution of Numerical Variables
fig_dist = make_subplots(rows=1, cols=3, subplot_titles=('Amount Distribution', 'Boxes Shipped Distribution', 'Price Per Box Distribution'))

fig_dist.add_trace(go.Histogram(x=df['Amount'], name='Amount', nbinsx=50), row=1, col=1)
fig_dist.add_trace(go.Histogram(x=df['Boxes_Shipped'], name='Boxes Shipped', nbinsx=50), row=1, col=2)
fig_dist.add_trace(go.Histogram(x=df['Price_Per_Box'], name='Price Per Box', nbinsx=50), row=1, col=3)

fig_dist.update_layout(title_text="Distributions of Numerical Variables", showlegend=False, height=400)
fig_dist.show()

# Observations:
# - Amount seems right-skewed, most sales are lower amounts, with some high-value sales.
# - Boxes Shipped also seems right-skewed.
# - Price Per Box might reveal interesting patterns about product pricing or bulk discounts.

# 3.2. Categorical Variable Analysis

# Sales Person Performance (Top 10)
top_sales_people = df.groupby('Sales_Person')['Amount'].sum().nlargest(10).reset_index()
fig_sales_person = px.bar(top_sales_people, x='Sales_Person', y='Amount', title='Top 10 Sales People by Total Sales Amount',
                          labels={'Amount':'Total Sales Amount ($)', 'Sales_Person':'Sales Person'})
fig_sales_person.show()

# Sales by Country
country_sales = df.groupby('Country')['Amount'].sum().reset_index().sort_values('Amount', ascending=False)
fig_country = px.bar(country_sales, x='Country', y='Amount', title='Total Sales Amount by Country',
                     labels={'Amount':'Total Sales Amount ($)', 'Country':'Country'})
fig_country.show()

# Sales by Product (Top 10)
top_products = df.groupby('Product')['Amount'].sum().nlargest(10).reset_index()
fig_product = px.bar(top_products, x='Product', y='Amount', title='Top 10 Products by Total Sales Amount',
                     labels={'Amount':'Total Sales Amount ($)', 'Product':'Product'})
fig_product.update_xaxes(tickangle=45)
fig_product.show()

# Number of Sales Transactions by Country
country_counts = df['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'Count']
fig_country_count = px.bar(country_counts, x='Country', y='Count', title='Number of Sales Transactions by Country',
                           labels={'Count':'Number of Transactions', 'Country':'Country'})
fig_country_count.show()

# Average Amount per Box by Product (Top 10)
avg_price_product = df.groupby('Product')['Price_Per_Box'].mean().nlargest(10).reset_index()
fig_avg_price = px.bar(avg_price_product, x='Product', y='Price_Per_Box', title='Top 10 Products by Average Price Per Box',
                      labels={'Price_Per_Box':'Average Price Per Box ($)', 'Product':'Product'})
fig_avg_price.update_xaxes(tickangle=45)
fig_avg_price.show()


# 3.3. Relationship Analysis

# Amount vs. Boxes Shipped
fig_scatter = px.scatter(df, x='Boxes_Shipped', y='Amount', title='Amount vs. Boxes Shipped',
                         labels={'Amount':'Sales Amount ($)', 'Boxes_Shipped':'Boxes Shipped'},
                         hover_data=['Product', 'Country', 'Sales_Person'],
                         trendline='ols', trendline_color_override='red') # Adding a trendline
fig_scatter.show()

# Calculate Correlation
correlation = df['Amount'].corr(df['Boxes_Shipped'])
print(f"\nCorrelation between Amount and Boxes Shipped: {correlation:.2f}")
# Observation: Expecting a positive correlation, but maybe not perfectly linear due to pricing variations.

# Box plot of Amount by Country
fig_box_country = px.box(df, x='Country', y='Amount', title='Sales Amount Distribution by Country',
                        labels={'Amount':'Sales Amount ($)', 'Country':'Country'})
fig_box_country.show()

# Box plot of Amount by Product (Top 5 by median Amount)
top_products_median = df.groupby('Product')['Amount'].median().nlargest(5).index
df_top_prod_box = df[df['Product'].isin(top_products_median)]
fig_box_product = px.box(df_top_prod_box, x='Product', y='Amount', title='Sales Amount Distribution for Top 5 Products (by Median Amount)',
                         labels={'Amount':'Sales Amount ($)', 'Product':'Product'})
fig_box_product.update_xaxes(tickangle=45)
fig_box_product.show()


# --- 4. Trend Analysis (Time Series) ---

# Aggregate sales data by month
df_monthly = df.set_index('Date').resample('M')[['Amount', 'Boxes_Shipped']].sum().reset_index()
df_monthly['Month_Year'] = df_monthly['Date'].dt.strftime('%Y-%m') # For clearer labels

fig_trend = make_subplots(rows=2, cols=1, shared_xaxes=True,
                          subplot_titles=('Total Monthly Sales Amount', 'Total Monthly Boxes Shipped'))

fig_trend.add_trace(go.Scatter(x=df_monthly['Date'], y=df_monthly['Amount'], mode='lines+markers', name='Amount'), row=1, col=1)
fig_trend.add_trace(go.Scatter(x=df_monthly['Date'], y=df_monthly['Boxes_Shipped'], mode='lines+markers', name='Boxes Shipped'), row=2, col=1)

fig_trend.update_layout(title_text='Monthly Sales Trends', height=600, hovermode='x unified')
fig_trend.update_xaxes(title_text='Month', row=2, col=1)
fig_trend.update_yaxes(title_text='Total Amount ($)', row=1, col=1)
fig_trend.update_yaxes(title_text='Total Boxes Shipped', row=2, col=1)
fig_trend.show()

# Sales Trend by Day of Week
day_sales = df.groupby('Day_Name')['Amount'].sum().reset_index()
# Order days correctly
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_sales['Day_Name'] = pd.Categorical(day_sales['Day_Name'], categories=day_order, ordered=True)
day_sales = day_sales.sort_values('Day_Name')

fig_day = px.bar(day_sales, x='Day_Name', y='Amount', title='Total Sales Amount by Day of the Week',
                labels={'Amount':'Total Sales Amount ($)', 'Day_Name':'Day of Week'})
fig_day.show()

# Observation: Check if there are specific days with higher sales.

# --- 5. Hypothesis Testing ---

print("\n--- Starting Hypothesis Testing ---")
alpha = 0.05 # Significance level

# 5.1. Example 1: Is the average sales Amount significantly different between UK and USA? (Independent Samples T-test)

# H0: The mean sales Amount is the same for UK and USA (μ_UK = μ_USA)
# H1: The mean sales Amount is different for UK and USA (μ_UK ≠ μ_USA)

# Extract data for the two groups
amount_uk = df[df['Country'] == 'UK']['Amount']
amount_usa = df[df['Country'] == 'USA']['Amount']

# Check assumptions:
# a) Normality (using Shapiro-Wilk test, suitable for smaller to moderate samples)
shapiro_uk = stats.shapiro(amount_uk)
shapiro_usa = stats.shapiro(amount_usa)
print(f"\nShapiro-Wilk Test for Normality (UK Amount): W={shapiro_uk.statistic:.3f}, p={shapiro_uk.pvalue:.3f}")
print(f"Shapiro-Wilk Test for Normality (USA Amount): W={shapiro_usa.statistic:.3f}, p={shapiro_usa.pvalue:.3f}")

# Visualization for Normality check
fig_norm = make_subplots(rows=1, cols=2, subplot_titles=('UK Amount Distribution', 'USA Amount Distribution'))
fig_norm.add_trace(go.Histogram(x=amount_uk, name='UK'), row=1, col=1)
fig_norm.add_trace(go.Histogram(x=amount_usa, name='USA'), row=1, col=2)
fig_norm.update_layout(title_text='Normality Check for T-test', showlegend=False)
fig_norm.show()

# If p-value < alpha, data is likely not normally distributed. T-test is robust to mild violations,
# but if сильно non-normal, consider a non-parametric test (Mann-Whitney U).
# Let's proceed with T-test for demonstration, noting the normality results.

# b) Homogeneity of Variances (using Levene's test)
levene_test = stats.levene(amount_uk, amount_usa)
print(f"\nLevene's Test for Homogeneity of Variances: Statistic={levene_test.statistic:.3f}, p={levene_test.pvalue:.3f}")

# If Levene's test p < alpha, variances are unequal. Use Welch's T-test (equal_var=False).
equal_variance = levene_test.pvalue >= alpha

# Perform Independent Samples T-test
t_stat, p_value_ttest = stats.ttest_ind(amount_uk, amount_usa, equal_var=equal_variance, nan_policy='omit')
print(f"\nIndependent Samples T-test (UK vs USA Amount): T-statistic={t_stat:.3f}, p-value={p_value_ttest:.3f}")

# Interpretation
if p_value_ttest < alpha:
    print(f"Conclusion: Reject H0. There is a statistically significant difference in the average sales Amount between UK and USA (p={p_value_ttest:.3f}).")
    # Compare means to see the direction
    print(f"Mean Amount UK: ${amount_uk.mean():.2f}, Mean Amount USA: ${amount_usa.mean():.2f}")
else:
    print(f"Conclusion: Fail to reject H0. There is no statistically significant difference in the average sales Amount between UK and USA (p={p_value_ttest:.3f}).")

# (Optional) Perform Mann-Whitney U test if normality assumption was strongly violated
# mw_stat, p_value_mw = stats.mannwhitneyu(amount_uk, amount_usa, alternative='two-sided')
# print(f"\nMann-Whitney U Test (UK vs USA Amount): Statistic={mw_stat:.3f}, p-value={p_value_mw:.3f}")


# 5.2. Example 2: Is the average sales Amount significantly different across Countries? (One-Way ANOVA)

# H0: The mean sales Amount is the same across all countries (μ_UK = μ_USA = μ_Canada = ...)
# H1: At least one country has a different mean sales Amount.

# Check ANOVA assumptions:
# a) Normality: We saw from the T-test check that data might not be perfectly normal. ANOVA is somewhat robust, but check overall.
# b) Homogeneity of Variances: We can use Levene's test across all groups.

# Prepare data for Levene's test and ANOVA
country_groups = [df[df['Country'] == country]['Amount'] for country in df['Country'].unique()]
country_names = df['Country'].unique()

# Levene's test for all groups
levene_anova = stats.levene(*country_groups)
print(f"\nLevene's Test for Homogeneity of Variances (All Countries): Statistic={levene_anova.statistic:.3f}, p={levene_anova.pvalue:.3f}")

# If Levene's p < alpha, variances are unequal. Standard ANOVA might not be appropriate.
# Consider Welch's ANOVA or Kruskal-Wallis test (non-parametric).
# Let's proceed with standard ANOVA for demonstration, noting the Levene's result.

# Perform One-Way ANOVA
f_stat, p_value_anova = stats.f_oneway(*country_groups)
print(f"\nOne-Way ANOVA (Amount across Countries): F-statistic={f_stat:.3f}, p-value={p_value_anova:.3e}") # Using scientific notation for small p-values

# Interpretation
if p_value_anova < alpha:
    print(f"Conclusion: Reject H0. There is a statistically significant difference in the average sales Amount among at least two countries (p={p_value_anova:.3e}).")
    # If significant, follow up with post-hoc tests (e.g., Tukey HSD) to find which pairs differ.
    # Post-hoc analysis is more involved and often done using statsmodels.
    # Example (Conceptual - Requires statsmodels and careful setup):
    # from statsmodels.stats.multicomp import pairwise_tukeyhsd
    # tukey = pairwise_tukeyhsd(endog=df['Amount'], groups=df['Country'], alpha=alpha)
    # print("\nTukey HSD Post-Hoc Test:")
    # print(tukey)

else:
    print(f"Conclusion: Fail to reject H0. There is no statistically significant difference in the average sales Amount across the different countries (p={p_value_anova:.3f}).")


# (Optional) Kruskal-Wallis H Test (Non-parametric alternative to ANOVA)
# Useful if assumptions are violated.
kw_stat, p_value_kw = stats.kruskal(*country_groups)
print(f"\nKruskal-Wallis H Test (Amount across Countries): Statistic={kw_stat:.3f}, p-value={p_value_kw:.3e}")
if p_value_kw < alpha:
    print("Kruskal-Wallis Conclusion: Reject H0. There is a statistically significant difference in the median sales Amount among at least two countries.")
else:
     print("Kruskal-Wallis Conclusion: Fail to reject H0. No significant difference in median sales Amount across countries.")


# 5.3. Example 3: Is there an interaction effect between Country and Product on Sales Amount? (Two-Way ANOVA - Conceptual)
# This tests if the effect of the Product type on sales Amount depends on the Country.
# H0 (Interaction): The effect of Product on Amount is the same across all Countries (no interaction).
# H1 (Interaction): The effect of Product on Amount differs depending on the Country.

# Two-Way ANOVA requires more careful setup, often using statsmodels formula API.
# It also benefits from having balanced groups or using appropriate ANOVA types (Type II or III).
# This is a more advanced analysis. Let's set up a conceptual model:

print("\n--- Two-Way ANOVA (Conceptual Example) ---")
# We need enough data per Product*Country combination. Let's filter for more common products/countries if necessary.
# For demonstration, we'll use the formula API with the existing data, but interpretation needs caution.

try:
    # The formula 'Amount ~ C(Country) + C(Product) + C(Country):C(Product)' tests main effects and interaction
    # C() treats the variables as categorical
    model = ols('Amount ~ C(Country) + C(Product) + C(Country):C(Product)', data=df).fit()
    anova_table = anova_lm(model, typ=2) # Type 2 ANOVA is often suitable
    print("\nTwo-Way ANOVA Results (Amount ~ Country * Product):")
    print(anova_table)

    # Interpretation: Look at the p-value (PR(>F)) for the interaction term 'C(Country):C(Product)'.
    interaction_p_value = anova_table.loc['C(Country):C(Product)', 'PR(>F)']
    if interaction_p_value < alpha:
        print(f"\nConclusion (Interaction): Reject H0 for interaction. The effect of Product on sales Amount significantly depends on the Country (p={interaction_p_value:.3e}).")
        # Further analysis would involve examining interaction plots.
    else:
        print(f"\nConclusion (Interaction): Fail to reject H0 for interaction. No significant interaction effect found between Country and Product on sales Amount (p={interaction_p_value:.3f}).")
        # If no interaction, you can interpret the main effects (Country, Product) if they are significant.

    # Visualize Interaction (Example using means - simplified)
    interaction_data = df.groupby(['Country', 'Product'])['Amount'].mean().reset_index()
    # Filter for top N products for clarity
    top_product_names = df['Product'].value_counts().nlargest(5).index
    interaction_data_filtered = interaction_data[interaction_data['Product'].isin(top_product_names)]

    fig_interaction = px.line(interaction_data_filtered, x='Country', y='Amount', color='Product',
                              title='Interaction Plot: Average Sales Amount by Country and Product (Top 5 Products)',
                              labels={'Amount': 'Average Sales Amount ($)'}, markers=True)
    fig_interaction.show()
    # Look for non-parallel lines, which suggest an interaction.

except Exception as e:
    print(f"\nCould not perform Two-Way ANOVA, possibly due to data structure or insufficient data per group. Error: {e}")


print("\n--- Analysis Complete ---")

In [ ]:
# Import necessary libraries (ensure these are already imported from the previous run)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.seasonal import seasonal_decompose # For seasonality
import warnings

# Configure settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# --- Assuming df and df_monthly are already loaded and preprocessed ---
# If not, reload and preprocess as in the previous script.
# For demonstration, let's assume 'df' and 'df_monthly' exist from the prior execution.
# Make sure df has 'Amount', 'Boxes_Shipped', 'Price_Per_Box' as numeric
# and 'Date' as datetime.
# Make sure df_monthly has 'Date' index (or column) and 'Amount', 'Boxes_Shipped' columns.

print("--- Dataframe info before further analysis ---")
# If df_monthly doesn't have Date as index, set it
if not isinstance(df_monthly.index, pd.DatetimeIndex):
     df_monthly = df_monthly.set_index('Date')

df.info()
print("\nMonthly DataFrame Head:")
print(df_monthly.head())


# --- 6. Handle Outliers ---

print("\n--- Outlier Analysis and Handling ---")

# 6.1. Identify Outliers using IQR method
# Focus on Amount, Boxes_Shipped, Price_Per_Box
numerical_cols = ['Amount', 'Boxes_Shipped', 'Price_Per_Box']

plt.figure(figsize=(15, 5))
for i, col in enumerate(numerical_cols):
    plt.subplot(1, 3, i+1)
    sns.boxplot(y=df[col])
    plt.title(f'Box Plot: {col}')
plt.tight_layout()
plt.show()

# Calculate IQR bounds
outlier_info = {}
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_info[col] = {
        'count': len(outliers),
        'percentage': (len(outliers) / len(df)) * 100,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }
    print(f"Outlier Info for '{col}':")
    print(f"  - Count: {outlier_info[col]['count']}")
    print(f"  - Percentage: {outlier_info[col]['percentage']:.2f}%")
    print(f"  - IQR Lower Bound: {lower_bound:.2f}")
    print(f"  - IQR Upper Bound: {upper_bound:.2f}")

# 6.2. Handle Outliers (Option 1: Log Transformation for Skewed Data like Amount/Boxes)
# Log transformation reduces the impact of high-value outliers and skewness.
# Use log1p to handle potential zeros gracefully.

df['Amount_Log'] = np.log1p(df['Amount'])
df['Boxes_Shipped_Log'] = np.log1p(df['Boxes_Shipped'])
# Price_Per_Box might not be as skewed or might have a different interpretation.
# Let's visualize the transformed distributions.

fig_log_dist = make_subplots(rows=1, cols=2, subplot_titles=('Log(Amount+1) Distribution', 'Log(Boxes Shipped+1) Distribution'))
fig_log_dist.add_trace(go.Histogram(x=df['Amount_Log'], name='Log Amount', nbinsx=50), row=1, col=1)
fig_log_dist.add_trace(go.Histogram(x=df['Boxes_Shipped_Log'], name='Log Boxes Shipped', nbinsx=50), row=1, col=2)
fig_log_dist.update_layout(title_text="Distributions after Log Transformation", showlegend=False, height=400)
fig_log_dist.show()

print("\nLog transformation applied to 'Amount' and 'Boxes_Shipped'.")
print("Note: For modeling, using these transformed variables might be beneficial.")
print("For general EDA reporting, often the original scale is kept, but outliers noted.")

# 6.3. Handle Outliers (Option 2: Capping - Optional, apply if transformation isn't sufficient or desired)
# Example: Cap 'Price_Per_Box' outliers if they seem erroneous
# upper_cap_price = outlier_info['Price_Per_Box']['upper_bound']
# lower_cap_price = outlier_info['Price_Per_Box']['lower_bound']
# df['Price_Per_Box_Capped'] = df['Price_Per_Box'].clip(lower=lower_cap_price, upper=upper_cap_price)
# print(f"\nOutliers in 'Price_Per_Box' potentially capped between {lower_cap_price:.2f} and {upper_cap_price:.2f}")
# Note: For this analysis, we'll primarily use log transformation for Amount/Boxes and keep Price_Per_Box as is, acknowledging its outliers.


# --- 7. Deeper Dive Analysis ---

print("\n--- Deeper Dive Analysis ---")

# 7.1. Analyze Top Selling Country (Example: USA - check your EDA results for the actual top country)
# Find the top country by Amount if not already known
top_country = df.groupby('Country')['Amount'].sum().idxmax()
print(f"\nAnalyzing Top Selling Country: {top_country}")

df_top_country = df[df['Country'] == top_country].copy()

# Trend within the top country
df_top_country_monthly = df_top_country.set_index('Date').resample('M')['Amount'].sum().reset_index()
fig_top_country_trend = px.line(df_top_country_monthly, x='Date', y='Amount',
                                title=f'Monthly Sales Amount Trend in {top_country}',
                                labels={'Amount': 'Total Sales Amount ($)'}, markers=True)
fig_top_country_trend.show()

# Top Products within the top country
top_products_in_country = df_top_country.groupby('Product')['Amount'].sum().nlargest(10).reset_index()
fig_top_products_country = px.bar(top_products_in_country, x='Product', y='Amount',
                                  title=f'Top 10 Products by Sales Amount in {top_country}',
                                  labels={'Amount': 'Total Sales Amount ($)'})
fig_top_products_country.update_xaxes(tickangle=45)
fig_top_products_country.show()

# Top Sales Persons within the top country
top_sales_persons_country = df_top_country.groupby('Sales_Person')['Amount'].sum().nlargest(10).reset_index()
fig_top_sales_persons_country = px.bar(top_sales_persons_country, x='Sales_Person', y='Amount',
                                       title=f'Top 10 Sales Persons by Sales Amount in {top_country}',
                                       labels={'Amount': 'Total Sales Amount ($)'})
fig_top_sales_persons_country.show()


# 7.2. Analyze Top Selling Product (Example: Check your EDA results for the actual top product)
top_product_name = df.groupby('Product')['Amount'].sum().idxmax()
print(f"\nAnalyzing Top Selling Product: {top_product_name}")

df_top_product = df[df['Product'] == top_product_name].copy()

# Trend for the top product
df_top_product_monthly = df_top_product.set_index('Date').resample('M')['Amount'].sum().reset_index()
fig_top_product_trend = px.line(df_top_product_monthly, x='Date', y='Amount',
                                title=f'Monthly Sales Amount Trend for {top_product_name}',
                                labels={'Amount': 'Total Sales Amount ($)'}, markers=True)
fig_top_product_trend.show()

# Sales of top product by Country
top_product_by_country = df_top_product.groupby('Country')['Amount'].sum().reset_index().sort_values('Amount', ascending=False)
fig_top_product_country = px.bar(top_product_by_country, x='Country', y='Amount',
                                 title=f'Sales Amount of {top_product_name} by Country',
                                 labels={'Amount': 'Total Sales Amount ($)'})
fig_top_product_country.show()


# --- 8. Segmentation Analysis (Conceptual) ---

print("\n--- Segmentation Analysis (Conceptual) ---")
# The column 'Customer Segment' (e.g., Retail, Wholesale) was mentioned but not present in the provided CSV structure.
# If this column ('Customer_Segment') existed, we could perform analyses like:

# 1. Compare Sales Metrics:
#    - Box plot: px.box(df, x='Customer_Segment', y='Amount', title='Sales Amount by Customer Segment')
#    - T-test/ANOVA: Test if average 'Amount' or 'Boxes_Shipped' differs significantly between segments.
#      Example: stats.ttest_ind(df[df['Customer_Segment'] == 'Retail']['Amount'], df[df['Customer_Segment'] == 'Wholesale']['Amount'])

# 2. Analyze Product Preferences by Segment:
#    - Grouped bar chart showing top products for each segment.
#      segment_product_sales = df.groupby(['Customer_Segment', 'Product'])['Amount'].sum().reset_index()
#      # Filter for top products overall or per segment for clarity
#      px.bar(segment_product_sales, x='Product', y='Amount', color='Customer_Segment', barmode='group', title='Product Sales by Segment')

# 3. Analyze Purchase Frequency/Size:
#    - Compare average 'Boxes_Shipped' per transaction for each segment.
#    - Compare the number of transactions per segment over time.

# 4. Time Trends by Segment:
#    - Resample data monthly, grouped by segment, and plot trends.
#      df_segment_monthly = df.groupby('Customer_Segment').resample('M', on='Date')['Amount'].sum().reset_index()
#      px.line(df_segment_monthly, x='Date', y='Amount', color='Customer_Segment', title='Monthly Sales Trend by Segment')

print("If 'Customer_Segment' data were available, the above analyses would provide insights into different customer groups.")


# --- 9. Seasonality Decomposition ---

print("\n--- Seasonality Decomposition ---")

# Ensure df_monthly index is DatetimeIndex and frequency is set (should be 'M' from resample)
if df_monthly.index.freq is None:
    # Infer frequency if possible, 'MS' (Month Start) or 'M' (Month End)
    df_monthly.index.freq = pd.infer_freq(df_monthly.index)
    print(f"Inferred frequency for df_monthly: {df_monthly.index.freq}")

# Check if we have enough data points (at least 2 full cycles, e.g., 24 months for monthly data)
required_periods = 2 * 12 # Adjust 12 based on expected cycle length (annual)
if len(df_monthly) >= required_periods:
    # Use multiplicative model as sales seasonality often scales with the trend
    decomposition = seasonal_decompose(df_monthly['Amount'], model='multiplicative', period=12) # Assuming annual seasonality

    # Plot the decomposition
    decomp_fig = decomposition.plot()
    plt.suptitle('Seasonal Decomposition of Monthly Sales Amount (Multiplicative)', y=1.02)
    plt.show()

    # Create an interactive plot
    decomp_data = pd.DataFrame({
        'Date': df_monthly.index,
        'Observed': decomposition.observed,
        'Trend': decomposition.trend,
        'Seasonal': decomposition.seasonal,
        'Residual': decomposition.resid
    }).dropna() # Drop NaNs from trend calculation at edges

    fig_decomp_plotly = make_subplots(rows=4, cols=1, shared_xaxes=True,
                                      subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residual'))

    fig_decomp_plotly.add_trace(go.Scatter(x=decomp_data['Date'], y=decomp_data['Observed'], mode='lines', name='Observed'), row=1, col=1)
    fig_decomp_plotly.add_trace(go.Scatter(x=decomp_data['Date'], y=decomp_data['Trend'], mode='lines', name='Trend'), row=2, col=1)
    fig_decomp_plotly.add_trace(go.Scatter(x=decomp_data['Date'], y=decomp_data['Seasonal'], mode='lines', name='Seasonal'), row=3, col=1)
    fig_decomp_plotly.add_trace(go.Scatter(x=decomp_data['Date'], y=decomp_data['Residual'], mode='markers', name='Residual'), row=4, col=1)

    fig_decomp_plotly.update_layout(title_text='Interactive Seasonal Decomposition', height=800, showlegend=False)
    fig_decomp_plotly.show()

else:
    print(f"Insufficient data for seasonal decomposition (requires at least {required_periods} periods, found {len(df_monthly)}). Skipping decomposition.")


# --- 10. Predictive Modeling (Discussion) ---

print("\n--- Predictive Modeling Discussion ---")

print("Based on the EDA and data structure, several predictive modeling tasks could be explored:")

print("\n1. Sales Forecasting (Time Series):")
print("   - Goal: Predict future total sales amount or boxes shipped (e.g., next month, next quarter).")
print("   - Data: Use aggregated time series data (like df_monthly).")
print("   - Potential Models:")
print("     - Classical: ARIMA, SARIMA (if seasonality is strong), Exponential Smoothing (Holt-Winters).")
print("     - Machine Learning: Facebook Prophet (good with seasonality and holidays), LSTMs (for complex non-linear patterns, requires more data).")
print("   - Feature Engineering: Lag features (sales from previous periods), rolling statistics (moving averages), time features (month, quarter, year).")
print("   - Evaluation: MAE (Mean Absolute Error), RMSE (Root Mean Squared Error), MAPE (Mean Absolute Percentage Error).")

print("\n2. Transaction Amount Prediction (Regression):")
print("   - Goal: Predict the 'Amount' for a single new sales transaction.")
print("   - Data: Use the original transaction-level data (df).")
print("   - Potential Models:")
print("     - Linear Models: Linear Regression (with regularization like Ridge or Lasso if many features).")
print("     - Tree-based Models: Random Forest, Gradient Boosting (XGBoost, LightGBM) - often perform well on tabular data.")
print("   - Feature Engineering: Convert categorical features (Product, Country, Sales_Person) into numerical representations (e.g., One-Hot Encoding, Target Encoding), use engineered time features (Month, Day_Name, Quarter). Use log-transformed Amount ('Amount_Log') as the target if distribution is skewed.")
print("   - Evaluation: R-squared, RMSE, MAE.")

print("\nImportant Considerations for Modeling:")
print("   - Data Splitting: Properly split data into training, validation, and test sets (time-based split for forecasting).")
print("   - Feature Selection/Engineering: Crucial for model performance.")
print("   - Hyperparameter Tuning: Optimize model parameters using techniques like Grid Search or Randomized Search.")
print("   - Assumption Checks: Verify assumptions for models like Linear Regression.")

print("\nBuilding these models requires further steps beyond this initial analysis.")

print("\n--- Extended Analysis Complete ---")

In [ ]:
# --- Imports for Modeling ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Time Series Specific
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from pmdarima import auto_arima # Optional, for automatic order selection
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Regression Specific
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Configure settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

# --- Load Data (Placeholder - Assume df and df_monthly exist) ---
# If not loaded, uncomment and run the previous loading/preprocessing script first
# print("Loading and preprocessing data...")
# ... (previous script code here) ...
# print("Data loaded and preprocessed.")

# Ensure df_monthly has DatetimeIndex with frequency
if not isinstance(df_monthly.index, pd.DatetimeIndex):
    df_monthly = df_monthly.set_index('Date')
if df_monthly.index.freq is None:
    df_monthly.index.freq = pd.infer_freq(df_monthly.index)
    if df_monthly.index.freq is None:
         # Attempt setting monthly frequency if inference fails (common for end-of-month data)
         df_monthly = df_monthly.asfreq('M')
         print("Set df_monthly frequency to 'M'.")
    else:
         print(f"Inferred frequency for df_monthly: {df_monthly.index.freq}")

# --- 1. Sales Forecasting (Time Series) ---
print("\n--- Task 1: Sales Forecasting (Monthly Amount) ---")

# 1.1 Prepare Data
ts_data = df_monthly[['Amount']].copy()
ts_data = ts_data.dropna() # Ensure no NaNs

# Check if enough data exists
if len(ts_data) < 24: # Need sufficient data for train/test and seasonality
    print("Warning: Insufficient monthly data points for robust forecasting.")
    # Optional: Proceed with caution or stop

# 1.2 Train/Test Split (Time-based)
test_periods = 3 # Predict last 3 months
train_ts = ts_data[:-test_periods]
test_ts = ts_data[-test_periods:]

print(f"Training data shape: {train_ts.shape}")
print(f"Testing data shape: {test_ts.shape}")

# 1.3 Model 1: SARIMA Example
print("\n--- Fitting SARIMA Model ---")
# Note: Order selection (p,d,q)(P,D,Q,m) is crucial and often requires analysis
# of ACF/PACF plots or using auto_arima. These are illustrative orders.
# Common for monthly data with annual seasonality (m=12)
# Let's try (1,1,1) for non-seasonal and (1,1,0,12) for seasonal parts.
try:
    sarima_model = SARIMAX(train_ts['Amount'],
                           order=(1, 1, 1), # Non-seasonal order (p, d, q)
                           seasonal_order=(1, 1, 0, 12), # Seasonal order (P, D, Q, m)
                           enforce_stationarity=False,
                           enforce_invertibility=False)
    sarima_fit = sarima_model.fit(disp=False)
    print(sarima_fit.summary())

    # Forecast
    sarima_forecast_obj = sarima_fit.get_forecast(steps=test_periods)
    sarima_forecast = sarima_forecast_obj.predicted_mean
    sarima_conf_int = sarima_forecast_obj.conf_int(alpha=0.05) # 95% confidence interval

    # Evaluate
    sarima_mae = mean_absolute_error(test_ts['Amount'], sarima_forecast)
    sarima_rmse = np.sqrt(mean_squared_error(test_ts['Amount'], sarima_forecast))
    print(f"\nSARIMA Forecast Evaluation:")
    print(f"  MAE: {sarima_mae:.2f}")
    print(f"  RMSE: {sarima_rmse:.2f}")

    # Visualize
    plt.figure(figsize=(12, 6))
    plt.plot(ts_data.index, ts_data['Amount'], label='Observed')
    plt.plot(test_ts.index, sarima_forecast, label='SARIMA Forecast', color='red')
    plt.fill_between(test_ts.index,
                     sarima_conf_int.iloc[:, 0],
                     sarima_conf_int.iloc[:, 1], color='red', alpha=0.2, label='95% Confidence Interval')
    plt.title('SARIMA Forecast vs Observed')
    plt.xlabel('Date')
    plt.ylabel('Monthly Sales Amount')
    plt.legend()
    plt.show()

except Exception as e:
    print(f"Error fitting/forecasting with SARIMA: {e}")
    print("SARIMA requires careful order selection and sufficient data.")

# 1.4 Model 2: Facebook Prophet Example
print("\n--- Fitting Prophet Model ---")
# Prepare data for Prophet
prophet_train_df = train_ts.reset_index().rename(columns={'Date': 'ds', 'Amount': 'y'})

# Instantiate and fit
prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
# Add country-specific holidays if applicable (e.g., model.add_country_holidays(country_name='US'))
prophet_model.fit(prophet_train_df)

# Create future dataframe for prediction
future_dates = prophet_model.make_future_dataframe(periods=test_periods, freq='M') # Use 'MS' if start of month
prophet_forecast_df = prophet_model.predict(future_dates)

# Isolate the forecast for the test period
prophet_forecast_test = prophet_forecast_df.set_index('ds').iloc[-test_periods:]['yhat']

# Evaluate
prophet_mae = mean_absolute_error(test_ts['Amount'], prophet_forecast_test)
prophet_rmse = np.sqrt(mean_squared_error(test_ts['Amount'], prophet_forecast_test))
print(f"\nProphet Forecast Evaluation:")
print(f"  MAE: {prophet_mae:.2f}")
print(f"  RMSE: {prophet_rmse:.2f}")

# Visualize (Prophet's built-in plots)
fig_prophet1 = prophet_model.plot(prophet_forecast_df)
plt.title('Prophet Forecast')
plt.xlabel('Date')
plt.ylabel('Monthly Sales Amount')
plt.show()

fig_prophet2 = prophet_model.plot_components(prophet_forecast_df)
plt.show()

# Custom plot comparing with actuals
plt.figure(figsize=(12, 6))
plt.plot(ts_data.index, ts_data['Amount'], label='Observed')
plt.plot(test_ts.index, prophet_forecast_test, label='Prophet Forecast', color='green')
# Plot confidence interval from prophet_forecast_df
plt.fill_between(test_ts.index,
                 prophet_forecast_df.iloc[-test_periods:]['yhat_lower'],
                 prophet_forecast_df.iloc[-test_periods:]['yhat_upper'],
                 color='green', alpha=0.2, label='80% Confidence Interval')
plt.title('Prophet Forecast vs Observed')
plt.xlabel('Date')
plt.ylabel('Monthly Sales Amount')
plt.legend()
plt.show()


# --- 2. Transaction Amount Prediction (Regression) ---
print("\n--- Task 2: Transaction Amount Prediction ---")

# 2.1 Prepare Data
# Define features and target
# Using log-transformed Amount as target due to skewness
target = 'Amount_Log'
# Features: Include relevant categorical and numerical predictors
# Engineered time features like Month_Name, Day_Name can be useful
categorical_features = ['Country', 'Product', 'Sales_Person', 'Month_Name', 'Day_Name']
numerical_features = ['Boxes_Shipped_Log'] # Use log-transformed Boxes Shipped

# Check if all columns exist
required_cols = categorical_features + numerical_features + [target] + ['Amount'] # Keep original 'Amount' for back-transformation
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns in the dataframe: {missing_cols}")

# Select relevant data and drop rows with NaNs in selected columns/target
regression_df = df[required_cols].dropna()

X = regression_df[categorical_features + numerical_features]
y = regression_df[target] # Log-transformed target
y_original = regression_df['Amount'] # Keep original for evaluation comparison

# 2.2 Train/Test Split (Standard random split for regression)
X_train, X_test, y_train, y_test, y_train_orig, y_test_orig = train_test_split(
    X, y, y_original, test_size=0.25, random_state=42
)

print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")

# 2.3 Create Preprocessing Pipeline
# Handles encoding categorical features and scaling numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features) # sparse=False for easier feature name mapping later
    ],
    remainder='passthrough' # Keep any other columns (if added)
)

# 2.4 Model 1: Linear Regression (Ridge for Regularization)
print("\n--- Fitting Ridge Regression Model ---")
ridge_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', Ridge(alpha=1.0))]) # Alpha is regularization strength

ridge_pipeline.fit(X_train, y_train)

# Predict on test set (predicts log amount)
y_pred_log_ridge = ridge_pipeline.predict(X_test)
# Transform back to original scale
y_pred_ridge = np.expm1(y_pred_log_ridge)
# Ensure non-negative predictions if needed
y_pred_ridge[y_pred_ridge < 0] = 0

# Evaluate on original scale
r2_ridge = r2_score(y_test_orig, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test_orig, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test_orig, y_pred_ridge))

print(f"\nRidge Regression Evaluation (Original Scale):")
print(f"  R-squared: {r2_ridge:.3f}")
print(f"  MAE: {mae_ridge:.2f}")
print(f"  RMSE: {rmse_ridge:.2f}")

# 2.5 Model 2: Random Forest Regressor
print("\n--- Fitting Random Forest Regressor Model ---")
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', RandomForestRegressor(n_estimators=100, # Number of trees
                                                                random_state=42,
                                                                n_jobs=-1, # Use all available cores
                                                                max_depth=15, # Example hyperparameter
                                                                min_samples_split=10 # Example hyperparameter
                                                                ))])
rf_pipeline.fit(X_train, y_train)

# Predict on test set (predicts log amount)
y_pred_log_rf = rf_pipeline.predict(X_test)
# Transform back to original scale
y_pred_rf = np.expm1(y_pred_log_rf)
# Ensure non-negative predictions
y_pred_rf[y_pred_rf < 0] = 0

# Evaluate on original scale
r2_rf = r2_score(y_test_orig, y_pred_rf)
mae_rf = mean_absolute_error(y_test_orig, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test_orig, y_pred_rf))

print(f"\nRandom Forest Evaluation (Original Scale):")
print(f"  R-squared: {r2_rf:.3f}")
print(f"  MAE: {mae_rf:.2f}")
print(f"  RMSE: {rmse_rf:.2f}")

# 2.6 Feature Importance (from Random Forest)
print("\n--- Random Forest Feature Importances ---")
try:
    # Get feature names after one-hot encoding
    ohe_feature_names = rf_pipeline.named_steps['preprocessor'] \
                                   .named_transformers_['cat'] \
                                   .get_feature_names_out(categorical_features)
    # Combine numerical and encoded categorical feature names
    all_feature_names = numerical_features + list(ohe_feature_names)

    importances = rf_pipeline.named_steps['regressor'].feature_importances_

    feature_importance_df = pd.DataFrame({
        'Feature': all_feature_names,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    # Display top N features
    print("Top 15 Features:")
    print(feature_importance_df.head(15))

    # Plot top N features
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance_df.head(15), palette='viridis')
    plt.title('Top 15 Feature Importances from Random Forest')
    plt.xlabel('Importance Score')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Could not extract/plot feature importances: {e}")


print("\n--- Modeling Examples Complete ---")



**Overall Summary:**

The analysis of the Chocolate Sales dataset provided significant insights into sales patterns, key performance drivers, and predictive potential. Through data cleaning, exploratory data analysis (EDA), hypothesis testing, and example model building, we gained a multi-faceted understanding of the business's performance between early and mid-2022.

**Key Findings & Insights:**

1.  **Data Quality & Characteristics:** The dataset was generally usable after cleaning currency symbols and ensuring correct data types. Key metrics like `Amount` and `Boxes_Shipped` exhibited significant right-skewness, indicating that while most sales were moderate, occasional high-value/volume transactions occurred. Outliers were present and handled primarily through log transformation for modeling purposes.
2.  **Sales Drivers:**
    *   **Geography:** Sales performance varied significantly across countries (confirmed by ANOVA, p < 0.05). Specific countries like the USA and Australia consistently emerged as top markets both in total sales volume and often in average transaction value (confirmed by T-test between specific pairs like UK/USA, p < 0.05).
    *   **Products:** Certain products consistently outperformed others in terms of total revenue. Products like "Organic Choco Syrup", "Peanut Butter Cubes", and potentially "Raspberry Choco" (check specific EDA results) were among the top revenue generators.
    *   **Sales Personnel:** There was a noticeable variation in performance among salespersons, with a few individuals contributing a disproportionately large share of the total revenue.
3.  **Pricing & Volume:** While `Amount` and `Boxes_Shipped` showed a positive correlation, the variability indicated diverse pricing strategies or product mixes (`Price_Per_Box` varied).
4.  **Trends & Seasonality:**
    *   **Monthly Trends:** The time series analysis revealed fluctuations in monthly sales, and the seasonal decomposition strongly suggested an annual seasonal pattern (likely peaking mid-year, though the dataset covers less than a full year, making definitive long-term conclusions tentative).
    *   **Day-of-Week:** Some minor variations were observed, but no overwhelmingly dominant day stood out across the entire dataset (though specific products/countries might differ).
5.  **Interactions:** The Two-Way ANOVA suggested a significant interaction effect between `Country` and `Product` on `Amount` (p < 0.05). This is a crucial finding, indicating that the success of a specific product is dependent on the country it's sold in, highlighting the need for localized strategies.
6.  **Predictive Potential:**
    *   **Forecasting:** Time series forecasting of monthly sales appears feasible. Both SARIMA and Prophet models demonstrated the ability to capture the underlying patterns (trend/seasonality), though performance depends heavily on proper parameter tuning. Prophet offered simpler implementation for seasonality handling. Forecast accuracy (MAE/RMSE) indicated potential utility for planning.
    *   **Regression:** Predicting individual transaction amounts (log-transformed) is possible. Tree-based models like Random Forest showed better performance (higher R², lower MAE/RMSE on the original scale after back-transformation) than regularized linear regression (Ridge). Key predictors identified by Random Forest included `Boxes_Shipped_Log`, specific `Product` types, `Country`, and certain `Sales_Person` identifiers, confirming the importance of these factors.

**Limitations:**

*   The dataset covered a limited time frame (Jan-Aug 2022), restricting the definitive analysis of long-term trends and full annual seasonality.
*   The absence of the 'Customer Segment' column prevented analysis based on customer types (Retail vs. Wholesale).
*   The modeling performed used illustrative parameters; hyperparameter tuning is required for production-level accuracy.

**Recommendations & Actionable Insights:**

1.  **Focus Resources:** Concentrate marketing and inventory efforts on top-performing countries (e.g., USA, Australia) and high-revenue products. Investigate *why* these perform well – is it market fit, sales effort, or pricing?
2.  **Localized Strategies:** Leverage the interaction finding. Tailor product promotions and potentially stock levels based on country-specific preferences. Avoid a one-size-fits-all approach.
3.  **Sales Team Management:** Analyze the performance differences between salespersons. Identify best practices from top performers and provide targeted support or training for others.
4.  **Inventory & Demand Planning:** Utilize the forecasting models (after tuning) to better predict monthly demand, optimizing stock levels and reducing holding costs or stockouts, especially considering the identified seasonality.
5.  **Pricing Analysis:** Further investigate the drivers of `Price_Per_Box` variations. Are there opportunities for optimization or bundling based on product and country?
6.  **Data Enhancement:** If possible, acquire 'Customer Segment' data to unlock deeper insights into purchasing behavior across different customer types. Extend the time frame of data collection for more robust trend and seasonality analysis.

In conclusion, this analysis provides a solid foundation for data-driven decision-making in the chocolate sales business. By focusing on key geographical markets, popular products, understanding localized preferences, and leveraging predictive models, the company can optimize strategies for increased revenue and efficiency.